# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df = city_data_df.set_index("City_ID")
# Display sample data
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,jamestown,42.0970,-79.2353,1.80,53,100,2.57,US,1680491401
1,avarua,-21.2078,-159.7750,27.03,69,75,3.60,CK,1680491491
2,castro,-24.7911,-50.0119,13.37,99,9,2.90,BR,1680491491
3,nedryhayliv,50.8340,33.8763,6.76,88,100,6.02,UA,1680491491
4,provideniya,64.3833,-173.3000,-10.91,67,75,4.02,RU,1680491492
...,...,...,...,...,...,...,...,...,...
566,hualmay,-11.0964,-77.6139,24.67,80,95,3.35,PE,1680491685
567,saint-joseph,-21.3667,55.6167,24.21,74,52,2.63,RE,1680491685
568,puerto baquerizo moreno,-0.9000,-89.6000,26.39,81,66,3.17,EC,1680491685


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    width = 1000,
    height = 600,
    size = 'Humidity',
    color = 'City'
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
reduced_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & 
                                        (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)].copy()

# Drop any rows with null values
reduced_city_data_df.dropna(inplace=True)

# Display sample data
reduced_city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
8,taoudenni,22.6783,-3.9836,26.73,7,0,3.60,ML,1680491494
19,north fort myers,26.6673,-81.8801,24.47,90,0,3.09,US,1680491419
59,manta,-0.9500,-80.7333,26.88,78,0,1.03,EC,1680491508
122,adrar,20.5022,-10.0711,22.86,6,0,3.22,MR,1680491521
166,saint-pierre,-21.3393,55.4781,24.82,65,0,3.60,RE,1680491538
334,tura,25.5198,90.2201,24.01,61,0,2.92,IN,1680491599
430,coxim,-18.5067,-54.7600,22.96,85,0,1.68,BR,1680491633
487,mandurah,-32.5500,115.7000,24.39,61,0,4.41,AU,1680491608
520,fuerte olimpo,-21.0415,-57.8738,23.12,61,0,1.76,PY,1680491669


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = reduced_city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
City_ID,,,,,,,,,,
8,taoudenni,22.6783,-3.9836,26.73,7,0,3.60,ML,1680491494,
19,north fort myers,26.6673,-81.8801,24.47,90,0,3.09,US,1680491419,
59,manta,-0.9500,-80.7333,26.88,78,0,1.03,EC,1680491508,
122,adrar,20.5022,-10.0711,22.86,6,0,3.22,MR,1680491521,
166,saint-pierre,-21.3393,55.4781,24.82,65,0,3.60,RE,1680491538,
334,tura,25.5198,90.2201,24.01,61,0,2.92,IN,1680491599,
430,coxim,-18.5067,-54.7600,22.96,85,0,1.68,BR,1680491633,
487,mandurah,-32.5500,115.7000,24.39,61,0,4.41,AU,1680491608,
520,fuerte olimpo,-21.0415,-57.8738,23.12,61,0,1.76,PY,1680491669,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 10 

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

###AskBCS Helped with this portion of the code###
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    Lat = row['Lat']
    Lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taoudenni - nearest hotel: No hotel found
north fort myers - nearest hotel: Red Roof Inn Ft Myers
manta - nearest hotel: La Gaviota
adrar - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
tura - nearest hotel: No hotel found
coxim - nearest hotel: Avendita
mandurah - nearest hotel: Forte Mandurah Quay Resort
fuerte olimpo - nearest hotel: No hotel found
ribas do rio pardo - nearest hotel: Hotel São Francisco


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
City_ID,,,,,,,,,,
8,taoudenni,22.6783,-3.9836,26.73,7,0,3.60,ML,1680491494,No hotel found
19,north fort myers,26.6673,-81.8801,24.47,90,0,3.09,US,1680491419,Red Roof Inn Ft Myers
59,manta,-0.9500,-80.7333,26.88,78,0,1.03,EC,1680491508,La Gaviota
122,adrar,20.5022,-10.0711,22.86,6,0,3.22,MR,1680491521,No hotel found
166,saint-pierre,-21.3393,55.4781,24.82,65,0,3.60,RE,1680491538,Tropic Hotel
334,tura,25.5198,90.2201,24.01,61,0,2.92,IN,1680491599,No hotel found
430,coxim,-18.5067,-54.7600,22.96,85,0,1.68,BR,1680491633,Avendita
487,mandurah,-32.5500,115.7000,24.39,61,0,4.41,AU,1680491608,Forte Mandurah Quay Resort
520,fuerte olimpo,-21.0415,-57.8738,23.12,61,0,1.76,PY,1680491669,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    width = 1000,
    height = 600,
    size = 'Humidity',
    color = 'City',
    hover_cols=['Hotel Name']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)